# Decoupling Simulation Accuracy from Mesh Quality

Simple python notebook to regenrate the teaser image.

The data and scripts to regenerate the other figures can be found 
[here](https://github.com/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality).

Note, we used [polyfem](https://polyfem.github.io) [C++ version](https://github.com/polyfem/polyfem/)  for all figures.

Importing libraries

In [7]:
import numpy as np
import polyfempy as pf
import json

#plotting:
import meshplot as mp
import plotly.offline as plotly
import plotly.graph_objs as go
#Necessary for the notebook
plotly.init_notebook_mode(connected=True)

Note that this notebook is designed to be run in the root folder.
If you want to reproduce the results of the teaser you can find the images [here](https://github.com/polyfem/Decoupling-Simulation-Accuracy-from-Mesh-Quality/tree/master/fig1-teaser/meshes).

Setup problem

In [2]:
settings = pf.Settings()

#Teaser uses laplacian
settings.set_pde(pf.PDEs.Laplacian)

#Use quartic function f = (2*y-0.9)^4+0.1 for bounary conditions
#and autodiff for rhs in C++, this problem is called Cubic (even if it quartic)

#The function is defined https://github.com/polyfem/polyfem/blob/d598e1764512da28d62ecbbbd1a832407ffb42de/src/problem/MiscProblem.cpp#L26
settings.problem = "Cubic"

First we solve using standard $P_1$ elements and our method which we enable with `settings.set_advanced_option("use_p_ref", True)`.

**Note** this takes some time, the last mesh quality is relly low!

In [4]:
n_meshes = 8

total_solutions = {}
total_errors = {}

for use_pref in [False, True]:
    if use_pref:
        settings.set_advanced_option("use_p_ref", True)
    
    solver = pf.Solver()
    solver.set_log_level(6)
    solver.settings(settings)

    solutions = []
    errors = []

    #iterating over input files, named from 0 to 8
    for i in range(n_meshes+1):
        solver.load_mesh_from_path("fig1-teaser/meshes/large_angle_strip_{}.obj".format(i), normalize_mesh=True)
        solver.solve()
        solver.compute_errors()

        #getting and storing solution
        v, f, sol = solver.get_sampled_solution()
        pts = np.append(v, sol, 1)
        solutions.append({"pts": pts, "f": f})

        #getting error:
        log = solver.get_log()
        log = json.loads(log)
        errors.append(log['err_l2'])
        
    #store these list in the total one
    total_solutions["ours" if use_pref else "p1"] = solutions
    total_errors["ours" if use_pref else "p1"] = errors

[2019-09-11 20:49:35.997] [geogram] [info] Loading file fig1-teaser/meshes/large_angle_strip_0.obj...
[2019-09-11 20:49:35.998] [geogram] [info] (FP64) nb_v:242 nb_e:0 nb_f:440 nb_b:42 tri:1 dim:3
[2019-09-11 20:49:35.998] [geogram] [info] Attributes on vertices: point[3]
[2019-09-11 20:49:36.011] [geogram] [info] Loading file fig1-teaser/meshes/large_angle_strip_1.obj...
[2019-09-11 20:49:36.011] [geogram] [info] (FP64) nb_v:263 nb_e:0 nb_f:480 nb_b:44 tri:1 dim:3
[2019-09-11 20:49:36.011] [geogram] [info] Attributes on vertices: point[3]
[2019-09-11 20:49:36.025] [geogram] [info] Loading file fig1-teaser/meshes/large_angle_strip_2.obj...
[2019-09-11 20:49:36.026] [geogram] [info] (FP64) nb_v:305 nb_e:0 nb_f:560 nb_b:48 tri:1 dim:3
[2019-09-11 20:49:36.026] [geogram] [info] Attributes on vertices: point[3]
[2019-09-11 20:49:36.040] [geogram] [info] Loading file fig1-teaser/meshes/large_angle_strip_3.obj...
[2019-09-11 20:49:36.040] [geogram] [info] (FP64) nb_v:389 nb_e:0 nb_f:720 nb_b

## Plot 3D results

In [5]:
#for nice plot we append the 2 meshses
def get_v_f_c(sols, i):
    #append points and offset x by 1.1
    pts_ours = np.array(total_solutions["ours"][i]["pts"])
    pts_p1 = np.array(total_solutions["p1"][i]["pts"])
    pts_p1[:, 0] += 1.1
    v = np.append(pts_ours, pts_p1, 0)
    
    #colors
    c = np.array(v[:, 2])
    
    #append faces
    f_ours = np.array(total_solutions["ours"][i]['f'])
    f_p1 = np.array(total_solutions["p1"][i]['f'])
    f_p1 += np.max(f_ours)+1
    f = np.append(f_ours, f_p1, 0)
    
    return v, f, c

Our results are on the left!

In [6]:
v, f, c = get_v_f_c(total_solutions, 0)
p = mp.plot(v, f, c, shading={"wireframe": False}, return_plot=True)

@mp.interact(mesh=[('mesh_{}'.format(i), i) for i in range(n_meshes+1)])
def ff(mesh):
    v, f, c = get_v_f_c(total_solutions, mesh)
    mp.plot(v, f, c, shading={"wireframe": False}, plot=p)
p

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.0499999…

interactive(children=(Dropdown(description='mesh', options=(('mesh_0', 0), ('mesh_1', 1), ('mesh_2', 2), ('mes…

## Error plor

In [27]:
colors = {'ours': 'rgb(85, 239, 196)', 'p1': 'rgb(255, 201, 26)'}
marker_shape = 'circle'
marker_size = 6

def get_plot_trace(errors, method):
    x = []
    y = []
    
    for i in range(len(errors[method])):
        x.append(i)
        y.append(errors[method][i])
        
    x, y = zip(*sorted(zip(x, y))[:])
    trace = go.Scatter(x=x, y=y,
        mode='lines+markers',
        name=method,
        line=dict(color=colors[method]),
        marker=dict(symbol=marker_shape, size=marker_size)
    )
    return trace

In [31]:
plot_data = [get_plot_trace(total_errors, "ours"), get_plot_trace(total_errors, "p1")]

layout = go.Layout(
    legend=dict(x=0.1, y=0.98),
    xaxis=dict(
        title="Mesh",
        nticks=5,
        ),
    yaxis=dict(
        title="Error L2",
        range=[0.0, 0.3]
    ),
    hovermode='closest')

fig = go.Figure(data=plot_data, layout=layout)
plotly.iplot(fig)